In [1]:
import numpy as np
import pandas as pd
import mysql.connector


In [2]:
#save key from keys.txt
with open('keys.txt') as f:
    lines = f.readlines()
    key = lines[0].strip()


In [3]:
db = mysql.connector.connect(
    host="localhost",
    user="ramsus",
    passwd=key,
    database="pagos"
)


In [4]:
mycursor = db.cursor()

('id', b'int', 'NO', 'PRI', None, 'auto_increment')
('paciente', b'varchar(35)', 'NO', '', None, '')
('valor', b'int', 'NO', '', None, '')
('actividad', b'varchar(35)', 'NO', '', None, '')
('fecha', b'timestamp', 'NO', '', None, '')
('nequi', b'tinyint(1)', 'NO', '', b'0', '')
('datafono', b'tinyint(1)', 'NO', '', b'0', '')
('transferencia', b'tinyint(1)', 'NO', '', b'0', '')
('efectivo', b'tinyint(1)', 'NO', '', b'0', '')
('notas', b'varchar(100)', 'YES', '', None, '')


In [ ]:
def print_sql(query):
    mycursor.execute(query)
    result = mycursor.fetchall()
    for x in result:
        print(x)

In [ ]:
print_sql("DESCRIBE pagos")

In [5]:
# read csv and normaliza rows names
excel = pd.read_csv('pagos.csv', encoding='latin-1', sep=',')
excel.columns = excel.columns.str.replace(' ', '')
del excel['%']
excel.dropna(inplace=True)
# convert Si in True and No in False
excel['Nequi'] = excel['Nequi'].map({'Si': 1, 'No': 0})
excel['Datafono'] = excel['Datafono'].map({'Si': 1, 'No': 0})
excel['Transferencia'] = excel['Transferencia'].map({'Si': 1, 'No': 0})
excel['Efectivo'] = excel['Efectivo'].map({'Si': 1, 'No': 0})
# convert value to number and remove the $ symbol and .
excel['Valor'] = excel['Valor'].str.replace('$', '')
excel['Valor'] = excel['Valor'].str.replace(',', '')
excel['Valor'] = excel['Valor'].str.replace(' ', '')
excel['Valor'] = excel['Valor'].astype(int)
# convert date in format day/month/year to datetime
excel['Fecha'] = pd.to_datetime(excel['Fecha'], format='%d/%m/%Y')

excel.reset_index(drop=True, inplace=True)

excel

/tmp/ipykernel_77031/4012513731.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  excel['Valor'] = excel['Valor'].str.replace('$', '')


,Fecha,Paciente,Valor,Actividad,Nequi,Datafono,Transferencia,Efectivo
0,2023-06-09,Manfry Ariza,60000,Valoracion,0,1,0,0
1,2023-06-09,Nazly Avila,185000,Montaje/retiro,1,0,0,0
2,2023-06-09,Paula Hernandez,25000,Radiografia,0,0,0,1
3,2023-06-09,Kevin Santiago Soto,105000,Control/Reparaciones,0,0,0,1
4,2023-06-09,Laura Cupa,60000,Control,0,1,0,0
...,...,...,...,...,...,...,...,...
134,2023-07-01,Carmen Mojica,50000,Control,0,0,0,1
135,2023-07-01,Carmen Mojica,10000,Control,0,0,1,0
136,2023-07-01,Lizeth Vargas,230000,"Papada, Encimas",0,0,0,1
137,2023-07-01,Maria Paula Correa,100000,Placa Activada,0,1,0,0


In [6]:
# Get dates already on the database
mycursor.execute(
    "SELECT DATE(fecha) AS dia FROM pagos GROUP BY dia ORDER BY dia DESC;")
dates = mycursor.fetchall()
dates = [x[0] for x in dates]

# Insert data of the new dates

for i in range(len(excel)):
    if excel['Fecha'][i].date() not in dates:
        mycursor.execute(
            "INSERT INTO pagos (paciente, valor, actividad, fecha, Nequi, Datafono, Transferencia, Efectivo) \
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
            (excel['Paciente'][i], int(
                excel['Valor'][i]), excel['Actividad'][i], excel['Fecha'][i],
             int(excel['Nequi'][i]), int(excel['Datafono'][i]),
             int(excel['Transferencia'][i]), int(excel['Efectivo'][i])))
        print(mycursor.rowcount, "record inserted.")


InternalError: 1213 (40001): Deadlock found when trying to get lock; try restarting transaction

In [ ]:
# check changes
print_sql("SELECT * FROM pagos WHERE id > 10 AND id < 120")


(1, 'Manfry Ariza', 60000, 'valoracion', datetime.datetime(2023, 6, 9, 0, 0), 0, 1, 0, 0, None)
(2, ' Nazly Avila ', 185000, 'montaje, retiro', datetime.datetime(2023, 6, 9, 0, 0), 1, 0, 0, 0, None)
(3, ' Paula Hernandez ', 25000, 'radiografia', datetime.datetime(2023, 6, 9, 0, 0), 0, 0, 0, 1, None)
(4, ' Kevin Santiago Soto ', 105000, 'control, reparaciones', datetime.datetime(2023, 6, 9, 0, 0), 0, 0, 0, 1, None)
(5, ' Laura Cupa ', 60000, 'control', datetime.datetime(2023, 6, 9, 0, 0), 0, 1, 0, 0, None)
(6, ' Nicole Daniela Benavides ', 300000, 'bichectomia', datetime.datetime(2023, 6, 9, 0, 0), 1, 0, 0, 0, None)
(7, ' Paula Camila Hernandez ', 50000, 'limpieza, braquets', datetime.datetime(2023, 6, 9, 0, 0), 0, 0, 0, 1, None)
(8, ' Paula Camila Hernandez ', 100000, 'limpieza, braquets', datetime.datetime(2023, 6, 9, 0, 0), 1, 0, 0, 0, None)
(9, ' Maria Ines ', 1000000, 'botox, hilos', datetime.datetime(2023, 6, 9, 0, 0), 0, 0, 0, 1, None)
(10, ' Nicol Daniela ', 180000, 'faja', date

In [ ]:
# Consultar los valores únicos de la columna de categorías en la base de datos
mycursor.execute("SELECT DISTINCT actividad FROM pagos")
result = mycursor.fetchall()


# Función para normalizar una categoría
def normalizar_categoria(categoria):
    categoria = categoria.replace("control caucho", "control, cauchos")
    return categoria

# Actualizar las categorías en la base de datos
for row in result:
    categoria = row[0]
    categoria_normalizada = normalizar_categoria(categoria)
    update_query = "UPDATE pagos SET actividad = %s WHERE actividad = %s"
    mycursor.execute(update_query, (categoria_normalizada, categoria))


In [ ]:
# Consultar las actividades que contienen "bichectomia" y "lipopapada" en el nombre
select_query = "SELECT * FROM pagos WHERE actividad LIKE '%bichectomia%' AND actividad LIKE '%lipopapada%'"
mycursor.execute(select_query)
result = mycursor.fetchall()

# Dividir las actividades y crear nuevos registros
for row in result:
    paciente = row[1]
    valor = row[2]
    actividad = row[3]
    fecha = row[4]
    nequi = row[5]
    datafono = row[6]
    transferencia = row[7]
    efectivo = row[8]
    print(paciente, valor, actividad, fecha, nequi, datafono, transferencia, efectivo)
    
    if "bichectomia" in actividad and "lipopapada" in actividad:
        bichectomia_value = 300000
        lipopapada_value = int(valor) - bichectomia_value
        
        # Crear un nuevo registro para "bichectomia"
        insert_query_bichectomia = "INSERT INTO pagos (paciente, fecha, nequi, datafono, transferencia, efectivo, actividad, valor) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
        mycursor.execute(insert_query_bichectomia, (paciente, fecha, nequi, datafono, transferencia, efectivo, "bichectomia", bichectomia_value))
        
        # Crear un nuevo registro para "lipopapada"
        insert_query_lipopapada = "INSERT INTO pagos (paciente, fecha, nequi, datafono, transferencia, efectivo, actividad, valor) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
        mycursor.execute(insert_query_lipopapada, (paciente, fecha, nequi, datafono, transferencia, efectivo, "lipopapada", lipopapada_value))

        # delete old record
        delete_query = "DELETE FROM pagos WHERE paciente = %s AND fecha = %s AND actividad = %s AND valor = %s"
        mycursor.execute(delete_query, (paciente, fecha, actividad, valor))

        



 Katherine Veloza  670000 lipopapada, bichectomia 2023-06-10 00:00:00 0 0 0 1
 Aura Yamile  780000 bichectomia, lipopapada 2023-06-10 00:00:00 0 1 0 0
 Yolima Montero  850000 bichectomia, lipopapada 2023-06-14 00:00:00 0 1 0 0


In [7]:
print_sql("SELECT DISTINCT actividad FROM pagos")

In [ ]:
# check changes
print_sql("SELECT * FROM pagos WHERE id > 138")

(139, ' Doctora Diana ', -595000, 'pago porcentaje', datetime.datetime(2023, 7, 1, 0, 0), 0, 0, 0, 1, None)
(158, ' Katherine Veloza ', 300000, 'bichectomia', datetime.datetime(2023, 6, 10, 0, 0), 0, 0, 0, 1, None)
(159, ' Katherine Veloza ', 370000, 'lipopapada', datetime.datetime(2023, 6, 10, 0, 0), 0, 0, 0, 1, None)
(160, ' Aura Yamile ', 300000, 'bichectomia', datetime.datetime(2023, 6, 10, 0, 0), 0, 1, 0, 0, None)
(161, ' Aura Yamile ', 480000, 'lipopapada', datetime.datetime(2023, 6, 10, 0, 0), 0, 1, 0, 0, None)
(162, ' Yolima Montero ', 300000, 'bichectomia', datetime.datetime(2023, 6, 14, 0, 0), 0, 1, 0, 0, None)
(163, ' Yolima Montero ', 550000, 'lipopapada', datetime.datetime(2023, 6, 14, 0, 0), 0, 1, 0, 0, None)


In [ ]:
# close connection
db.commit()
db.close()